In [1]:
# 13.07.23 BG - Dateiname nach oben + Symbol Dazu 
# 11.07.23 BG - leeres Datum bei MULTI wird korrekt abgebildet 
# 11.07.23 BG - Mit Ausgabe Dateinamen
# 09.07.23 BG - Acount ID weg 
# 06.07.23 BG - bessere Namensgebung 
# 06.07.23 BG - underlyingSymbol ergänzt 
# 05.07.23 BG - levelOfDetail ergänzt um später bei der Auswertung nach BaseCurrency filtern zu können
# 04.07.23 BG - currency -> IBcurrency 
# 22.06.23 Bg - Ausgabe mit +-Zeichen 
# 21.06.23 BG - Version 1.1. - Feldnamen sidn ale OK 
# 18.06.23 BG - amount fehlte als Feld 
# 15.06.23 BG - Version 1.0 - Fertig, Tabelle wird erfolgreich gefüllt 
# 15.06.23 BG - Neu gemacht, da irgend was weg geworfen 
# 12.06.23 BG - erster Wurf
#
# 
# verwendeter Report: Statment of Funds - Kapitalfluss Report 
#
# Infos zu dem Report 
# https://ibkrguides.com/reportingreference/reportguide/statement%20of%20fundsfq.htm
#

#------------ Diese Datei wird eingelesen 

#dateiname = "Statements_LX_2023-01_02.xml"
#dateiname = "Statements.xml"
#dateiname = "GmbH-Statements.xml"
dateiname = "2023_data.xml"

# --------------------------
#import locale
from babel.dates import format_date, format_datetime, format_time
import pymysql
from xml.dom.minidom import parse
from datetime import datetime, date, time

#import pyodbc  # Datenbankanbindung an Access 

#locale.setlocale(locale.LC_ALL, 'de_DE')
#locale.setlocale(locale.decimal_point, ',' )
#print (locale.localeconv())



In [2]:

def insertStatements(dbCursor, statement):
    global lastdate

    transactionID = statement.getAttribute("transactionID")
    if transactionID != "":
        transactionID = int(transactionID)
    else:
        transactionID = 0

    accountID = statement.getAttribute("accountId")

    settledate = statement.getAttribute("settleDate")
    date = statement.getAttribute("date")
    if settledate == "MULTI":
        date = lastdate
    else:
        lastdate = date

    formatting = "%Y%m%d"
    datum = datetime.strptime(date, formatting)

    amount = float(statement.getAttribute("amount"))

    IBcurrency = statement.getAttribute("currency")
    levelOfDetail = statement.getAttribute("levelOfDetail")

    balance = float(statement.getAttribute("balance"))
    balance = round(balance, 2)

    description = statement.getAttribute("description")
    activityCode = statement.getAttribute("activityCode")
    activityDescription = statement.getAttribute("activityDescription")

    symbol = statement.getAttribute("symbol")
    underlyingSymbol = statement.getAttribute("underlyingSymbol")

    # In the MySQL version, we will use parameterized queries to prevent SQL injection
    sql = "INSERT INTO statements (transactionID, accountID, datum, amount, IBcurrency, levelOfDetail, balance, description, activityCode, activityDescription, symbol, underlyingSymbol) "
    sql += "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    data = (transactionID, accountID, datum, amount, IBcurrency, levelOfDetail, balance, description, activityCode, activityDescription, symbol, underlyingSymbol)

    dbCursor.execute(sql, data)


In [3]:
# --------- Start des Programms --------------------------
lastdate = None 

# Verbindung zur Datenbank herstellen
dbConn = pymysql.Connect(
    host='localhost',
    port=3306,
    user='root',
    password='root',
    db='IB',
    charset='utf8'
)

dbCursor = dbConn.cursor()

In [4]:
#Delete database
sql= "DELETE FROM statements"
dbCursor.execute(sql) 

514

In [5]:
# Insert statements
domobject = parse(dateiname)
elementname = 'StatementOfFundsLine'

print (f"Start mit {dateiname}")

i = 0 ; 
limit = 10000 #100  
for statement in domobject.getElementsByTagName(elementname):
    #Insert into database
    insertStatements(dbCursor, statement)

    i = i+1
    if i % limit == 0  :
        print ("")
        print (i, end="")
    print("+", end="")

print ("")
print ( "Anzahl Datensätze eingetragen :" , i ) 

# Änderungen in der Datenbank speichern
dbConn.commit()
# Verbindung schließen
dbConn.close()

print(f"Fertig mit {dateiname}")


Start mit 2023_data.xml
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Anzahl Datensätze eingetragen : 514
Fertig mit 2023_data.xml
